In [1]:
include("saltsolver.jl")

smallest_stability_eigs (generic function with 1 method)

In [3]:
println("C3v, uniform everything")

N = 3
ℓ = 1
σ = 0.1
γ⟂ = 0.2
ωa = 4.0
D = 0.2
ɛ = (1+im*σ) * ones(N)
F = ones(N)
las = Laser(ɛ, F, ωa, γ⟂)

L = 1.0
h = L/N
Ering, ωring = ring_eigenpair(N, ℓ, h)
md = PassiveMode(copy(Ering), ωring)
laplacian!(J) = periodic!(J, h)
Dt = passive_threshold!(laplacian!, md, las, (1.0, 1.2))
Dt, imag(md.ω)

C3v, uniform everything


(0.9332690479444662,9.3237349405047e-11)

In [27]:
α = (γ⟂/σ - ωa)/3
β = α^3 - γ⟂*ωring^2/(2σ)
v = (-β + √(β^2-α^6))^(1/3)
ω = v + α^2/v - α
ωring^2 - ω^2*(1 + σ/γ⟂*(md.ω-ωa))

1.6571632954764937e-10 - 9.767512843977424e-10im

In [2]:
PassiveMode(E::AbstractVector, ω::Real) = Mode(E, ω+0.0im, 0.0)

function passive_threshold!{T<:Real}(laplacian!::Function, md::Mode, las::Laser, Ds::Tuple{T, T};
                    init_incr=0.05, tol=1e-10, maxits=10, newtontol=1e-8, newton_maxits=9, isprint=false)
    func(D) = begin
        passive_solve!(laplacian!, md, las, D, tol=newtontol, maxits=newton_maxits)
        imag(md.ω)
    end
    root(func, Ds, tol=tol, maxits=maxits, isprint=isprint)
end    

function passive_update!(md::Mode, dv::AbstractVector)
    md.ω  += dv[end]
    N = length(md.E)
    for i=1:N
        md.E[i] += dv[i]
    end
end

function passive_derivative!(md::Mode, las::Laser, D::Real, J::AbstractMatrix)
    N = length(md.E)
    J[end, md.imax] = 1
    
    ∂f∂ω  = sub(J, 1:N, N+1)
    γ = las.γ⟂ / (md.ω - las.ωa + im*las.γ⟂)
    ω²D∂γ∂ω = md.ω^2*D*(-γ^2/las.γ⟂)
    for i=1:N
        # columns must be set using =
        H = 1
        ∂M∂ω = 2md.ω*(las.ɛ[i]+D*γ*las.F[i]*H) + ω²D∂γ∂ω*las.F[i]*H
        ∂f∂ω[i]   = ∂M∂ω * md.E[i]
    end
end

function passive_salt_operator!(md::Mode, las::Laser, D::Real, J::AbstractMatrix)
    γ = las.γ⟂ / (md.ω - las.ωa + im*las.γ⟂)
    N = length(md.E)    
    for i=1:N
        H = 1
        # limitation: no hole-burning of other lasing mode
        z = md.ω^2 * (las.ɛ[i] + D*γ*H*las.F[i])
        J[i,i] +=  z
    end    
end

function passive_residual!(laplacian!::Function, md::Mode, las::Laser, D::Real, J::AbstractMatrix, f::AbstractVector)
    fill!(J, 0.0)
    fill!(f, 0.0)    
    N = length(md.E)
    laplacian!(sub(J, 1:N, 1:N))
    passive_salt_operator!(md, las, D, J)
    
    A_mul_B!(sub(f, 1:N), sub(J,1:N,1:N), md.E)
    f[end]   = md.E[md.imax] - 1
    norm(f)
end

function passive_solve!(laplacian!::Function, md::Mode, las::Laser, D::Real;
                maxits=9, tol=1e-8, isprint=false)
    N = length(md.E)
    isa(md.ω, Complex) || error("incorrect type of mode")
    length(las.ɛ) == N || error("incorrect length(ɛ)")
    J = zeros(Complex{Float64}, N+1, N+1)
    f = zeros(Complex{Float64}, N+1)
    
    for its=0:maxits
        res = passive_residual!(laplacian!, md, las, D, J, f)
        isprint && println("|f| = ", res)
        res < tol && return (res, its)
        its < maxits || throw(MaxIterationsReached())
        passive_derivative!(md, las, D, J)
        dv = scale!(J\f, -1)
        passive_update!(md, dv)
    end
end

passive_solve! (generic function with 1 method)

In [1]:
ℓ = 3
npiece = 4
Lcav = 1.0
wantcnv = true

nsym = 4ℓ
N = nsym * npiece
h = Lcav/N
Ering, ωring = ring_eigenpair(N, ℓ, h)
γ⟂ = 0.8
ωa = round(ωring)

ctrand(n) = rand(n)-0.5
ɛrpiece = 1 + 0.3*ctrand(npiece)
ɛipiece = 1.5 * rand(npiece) # want positive
Fpiece = 1 + 0.4*ctrand(npiece)

cnvize!(v) = (v[:] = (v + ring_flip(v))/2)
if wantcnv
    cnvize!(ɛrpiece)
    cnvize!(ɛipiece)
    cnvize!(Fpiece)
end

ɛ = cn_profile(ɛrpiece + im*ɛipiece, nsym)
F = cn_profile(Fpiece, nsym)
las = Laser(ɛ, F, ωa, γ⟂)
laplacian!(J) = periodic!(J, h)

md = Mode(copy(Ering), ωring, 0.0)
Dguesses = (0.6, 0.8)
Dt = threshold!(laplacian!, md, las, Dguesses)
ωt = md.ω
Es = (copy(md.E), ring_flip(md.E))

H, Is, Js, Ks, GD = overlap_integrals(Es, ωt, Dt, las, nsym, ℓ, Lcav);
I, J, K = Is[1], Js[1], Ks[1];
i, j, k = I/H, J/H, K/H;

println("H = ", H, '\n')

# for z² in (1, -1)
# #     println(2imag(2i+j + z²*k))
#     radicand = imag(j - z²*k)^2 + 8real(k)*real(j + z²*k)
#     term = -imag(j + 3z²*k)
#     println("term = ", term)
#     println("√rad = ", √radicand)
#     for sn in (1, -1)
#         println(term + sn*√radicand)
#     end
#     z² == 1 && println()
# end

LoadError: LoadError: UndefVarError: ring_eigenpair not defined
while loading In[1], in expression starting on line 9

In [5]:
isa(1, Integer)

true